In [1]:
import ee
ee.Authenticate()
ee.Initialize(project="ee-peisz2001")

In [2]:
roi   = ee.Geometry.Rectangle([-75, 40, -72, 41])
start = ee.Date('2024-01-01')
end   = ee.Date('2024-07-01')

In [3]:
bands = [
    'temperature_2m',
    'lake_mix_layer_temperature',
    'lake_shape_factor',
    'snowfall',
    'snowmelt',
    'runoff',
    'surface_runoff',
    'sub_surface_runoff',
    'evaporation_from_bare_soil',
    'evaporation_from_open_water_surfaces_excluding_oceans',
    'evaporation_from_the_top_of_canopy',
    'evaporation_from_vegetation_transpiration',
    'skin_reservoir_content',
    'u_component_of_wind_10m',
    'v_component_of_wind_10m',
    'surface_pressure',
    'total_precipitation',
    'leaf_area_index_high_vegetation',
    'leaf_area_index_low_vegetation'
]



In [ ]:
# https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_LAND_HOURLY

collection = (
    ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY')
      .filterDate(start, end)
      .filterBounds(roi)
      .select(bands)
      .map(lambda img: img.clip(roi).toFloat())
)

count = collection.size().getInfo()
server_list = collection.toList(count)

for i in range(count):
    img = ee.Image(server_list.get(i))
    ts  = img.date().format('YYYYMMdd_HH').getInfo()
    desc = f'ERA5_temp_{ts}'

    task = ee.batch.Export.image.toDrive(
        image=img,
        description=desc,
        folder='GEE_ERA5/1',
        fileNamePrefix=desc,
        region=roi.getInfo()['coordinates'],
        scale=1000,
        maxPixels=1e13
    )
    task.start()
print('All tasks started.')

All tasks started.
